# Day 13

## Part 1

- compare 2 signals
- first is `left` then `right`
- if both values are `int`:
    - the smaller value should be first
- if both values are `list`:
    - compare each value in turn as above
    - if `left` is shorter than `right` this is correct
- if one is `int` and one is `list` convert the `int` to a single item `list` and compare
- packets are enumerated from 1

`Determine which pairs of packets are already in the right order. What is the sum of the indices of those pairs?`

In [38]:
from utils import parse_from_file, ParseConfig

def parse_recursive_lists(string: str) -> 'list[list]':
    """
    returns a multi-dimensional list object of the string passed
    """
    output = list()
    segments = string.split(',')
    for index, segment in enumerate(segments):
        if segment.startswith('['):
            start = len(''.join(segments[:index])) + index
            print('start', string[:start + 1])
            level = 1
            end = start
            while level > 0 or end == start:
                # print(level, end, string[end])
                end += 1
                if string[end] == '[':
                    level += 1
                elif string[end] == ']':
                    level -= 1
            print('substring', string[start:end])
            output.append(parse_recursive_lists(string[start + 1:end]))
        else:
            output.append(int(segment.strip(']')))
    return output


test_string = '[[10,[],[9,8,6,[2,7,0,8]],[],[[4,6,2,1,4],6,5]]]'
print(parse_recursive_lists(test_string))


start [
substring [[10,[],[9,8,6,[2,7,0,8]],[],[[4,6,2,1,4],6,5]]
start [
substring [10,[],[9,8,6,[2,7,0,8]],[],[[4,6,2,1,4],6,5]
start 10,[
substring [


ValueError: invalid literal for int() with base 10: ''

In [ ]:

parser = ParseConfig('\n\n', ParseConfig('\n', ))

signal_pairs = parse_from_file('day_13.txt', parser)

print(signal_pairs[:10])